In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

train_df = pd.read_csv("/content/drive/My Drive/Text Analytics/drugsComTrain_raw.csv")
test_df = pd.read_csv("/content/drive/My Drive/Text Analytics/drugsComTest_raw.csv")

dataset_orig = pd.concat([train_df, test_df], ignore_index=True)

print(dataset_orig.head(20))


    uniqueID                            drugName  \
0     206461                           Valsartan   
1      95260                          Guanfacine   
2      92703                              Lybrel   
3     138000                          Ortho Evra   
4      35696            Buprenorphine / naloxone   
5     155963                              Cialis   
6     165907                      Levonorgestrel   
7     102654                        Aripiprazole   
8      74811                              Keppra   
9      48928  Ethinyl estradiol / levonorgestrel   
10     29607                          Topiramate   
11     75612                      L-methylfolate   
12    191290                             Pentasa   
13    221320                    Dextromethorphan   
14     98494                           Nexplanon   
15     81890                         Liraglutide   
16     48188                        Trimethoprim   
17    219869                       Amitriptyline   
18    212077

In [4]:
# prompt: show the basic information of data

print(dataset_orig.info())
print(dataset_orig.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215063 entries, 0 to 215062
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueID     215063 non-null  int64 
 1   drugName     215063 non-null  object
 2   condition    213869 non-null  object
 3   review       215063 non-null  object
 4   rating       215063 non-null  int64 
 5   date         215063 non-null  object
 6   usefulCount  215063 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 11.5+ MB
None
            uniqueID         rating    usefulCount
count  215063.000000  215063.000000  215063.000000
mean   116039.364814       6.990008      28.001004
std     67007.913366       3.275554      36.346069
min         0.000000       1.000000       0.000000
25%     58115.500000       5.000000       6.000000
50%    115867.000000       8.000000      16.000000
75%    173963.500000      10.000000      36.000000
max    232291.000000      10.000000    1291.0000

In [5]:
# check missing value

print(dataset_orig.isnull().sum())

uniqueID          0
drugName          0
condition      1194
review            0
rating            0
date              0
usefulCount       0
dtype: int64


In [6]:
#Create new dataset

dataset = dataset_orig.dropna().reset_index(drop=True)
print(dataset.isnull().sum())
print(dataset.info())

uniqueID       0
drugName       0
condition      0
review         0
rating         0
date           0
usefulCount    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213869 entries, 0 to 213868
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueID     213869 non-null  int64 
 1   drugName     213869 non-null  object
 2   condition    213869 non-null  object
 3   review       213869 non-null  object
 4   rating       213869 non-null  int64 
 5   date         213869 non-null  object
 6   usefulCount  213869 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 11.4+ MB
None


In [7]:
# delete the variables in condition which contains </span>

dataset = dataset[~dataset['condition'].astype(str).str.contains('</span>', na=False)]

print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
Index: 212698 entries, 0 to 213868
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueID     212698 non-null  int64 
 1   drugName     212698 non-null  object
 2   condition    212698 non-null  object
 3   review       212698 non-null  object
 4   rating       212698 non-null  int64 
 5   date         212698 non-null  object
 6   usefulCount  212698 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 13.0+ MB
None


In [8]:
dataset_clean = dataset.dropna().reset_index(drop=True)

print(dataset_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212698 entries, 0 to 212697
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueID     212698 non-null  int64 
 1   drugName     212698 non-null  object
 2   condition    212698 non-null  object
 3   review       212698 non-null  object
 4   rating       212698 non-null  int64 
 5   date         212698 non-null  object
 6   usefulCount  212698 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 11.4+ MB
None


In [9]:
# Removing HTML tags

import re

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

dataset_clean['review'] = dataset_clean['review'].apply(remove_html_tags)
dataset_clean['condition'] = dataset_clean['condition'].apply(remove_html_tags)


In [10]:
#Removal of special characters (only letters, numbers, punctuation)

import string

def remove_special_characters(text):
    # Define allowed characters (letters, numbers, punctuation)
    allowed_chars = string.ascii_letters + string.digits + string.punctuation + " "

    # Remove characters not in the allowed set
    cleaned_text = "".join(char for char in text if char in allowed_chars)
    return cleaned_text


dataset_clean['review'] = dataset_clean['review'].apply(remove_special_characters)
dataset_clean['condition'] = dataset_clean['condition'].apply(remove_special_characters)


In [11]:
# Remove all illegal characters

import re

def remove_illegal_chars(text):
    cleaned_text = re.sub(r'[^\w\s.,!?;:\-–—]', '', text)
    return cleaned_text

dataset_clean['review'] = dataset_clean['review'].apply(remove_illegal_chars)
dataset_clean['condition'] = dataset_clean['condition'].apply(remove_illegal_chars)


In [12]:
# Remove extra space
def remove_extra_space(text):
    return " ".join(text.split())

dataset_clean['review'] = dataset_clean['review'].apply(remove_extra_space)
dataset_clean['condition'] = dataset_clean['condition'].apply(remove_extra_space)


In [13]:
# Change all to lowercase

dataset_clean['review'] = dataset_clean['review'].str.lower()
dataset_clean['condition'] = dataset_clean['condition'].str.lower()
dataset_clean['drugName'] = dataset_clean['drugName'].str.lower()


In [14]:
# Stopwords Removal

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

dataset_clean['review'] = dataset_clean['review'].apply(remove_stopwords)
dataset_clean['condition'] = dataset_clean['condition'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
# Drop more stopwords
# Define custom stopwords (same ones that gave you best results)

custom_stopwords = set(stopwords.words('english')).union({
    "i039", "v", "amp", "use", "take", "get", "like", "work",
    "year", "month", "week", "day", "first", "one", "two", "three",
    "also", "since", "may", "could", "new",
    "im", "ive", "dont", "didnt", "doesnt", "wasnt", "arent", "isnt",
    "couldnt", "shouldnt", "wouldnt", "youre", "theyre", "weve", "youve",
    "doctor", "said", "told", "know", "see", "took", "read", "would", "still", "got"
})

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in custom_stopwords]
    return " ".join(filtered_words)

dataset_clean['review'] = dataset_clean['review'].apply(remove_stopwords)
dataset_clean['condition'] = dataset_clean['condition'].apply(remove_stopwords)

In [17]:
# delete the blank in the dataset_clean

dataset_clean = dataset_clean[dataset_clean['review'] != ""]
dataset_clean = dataset_clean[dataset_clean['condition'] != ""]

#Reset the index after removing rows
dataset_clean = dataset_clean.reset_index(drop=True)

# Delete the columns not use (uniqueid and date columns)
dataset_clean = dataset_clean.drop(columns=['uniqueID'])
dataset_clean = dataset_clean.drop(columns=['date'])

In [18]:
drug_final= dataset_clean
print(drug_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212675 entries, 0 to 212674
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   drugName     212675 non-null  object
 1   condition    212675 non-null  object
 2   review       212675 non-null  object
 3   rating       212675 non-null  int64 
 4   usefulCount  212675 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 8.1+ MB
None


In [19]:
# Export the DataFrame to an Excel file
drug_final.to_excel('/content/drive/My Drive/Text Analytics/drug_TM.xlsx', index=False)
